# Data Cleanising 

intialsing spark session

In [0]:
!pip install pyspark==3.1.2 --user
!pip install findspark --user

In [0]:
import findspark
findspark.init()

In [0]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [0]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

import required libraries

In [0]:
from pyspark.sql.functions import col

Loading Dataset fom CSV file to Spark DataFrame

In [0]:
path = "dbfs:/FileStore/tables/diabetes.csv"
df = spark.read.csv(path,header=True,inferSchema=True)
df.show()

+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck| BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|         0.0|     0|     1.0|        1|15.0|   1.0|   0.0|                 0.0|           0|     1|      1|                0|            1|        0.0|    5.0|    10.0|    20.0|     0.0|  0| 11|      4.0|   5.0|
|         2.0|     1|     0.0|        1|28.0|   0.0|   0.0|                 1.0|           0|     1|      0|                0|            1|        

All values should be intergers. But according to inital data analysis there are some floal values. First we will convert them to intergers



In [0]:
for col_name,col_type in df.dtypes:
    if col_type == 'double':
        df=df.withColumn(col_name,col(col_name).cast('int'))

df.show()

+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck|BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|           0|     0|       1|        1| 15|     1|     0|                   0|           0|     1|      1|                0|            1|          0|      5|      10|      20|       0|  0| 11|        4|     5|
|           2|     1|       0|        1| 28|     0|     0|                   1|           0|     1|      0|                0|            1|          0| 

In [0]:
df.printSchema()

root
 |-- Diabetes_012: integer (nullable = true)
 |-- HighBP: integer (nullable = true)
 |-- HighChol: integer (nullable = true)
 |-- CholCheck: integer (nullable = true)
 |-- BMI: integer (nullable = true)
 |-- Smoker: integer (nullable = true)
 |-- Stroke: integer (nullable = true)
 |-- HeartDiseaseorAttack: integer (nullable = true)
 |-- PhysActivity: integer (nullable = true)
 |-- Fruits: integer (nullable = true)
 |-- Veggies: integer (nullable = true)
 |-- HvyAlcoholConsump: integer (nullable = true)
 |-- AnyHealthcare: integer (nullable = true)
 |-- NoDocbcCost: integer (nullable = true)
 |-- GenHlth: integer (nullable = true)
 |-- MentHlth: integer (nullable = true)
 |-- PhysHlth: integer (nullable = true)
 |-- DiffWalk: integer (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Income: integer (nullable = true)



We can see all features converted to intergers. Now we can drop AnyHealthcare,MentHlth and PhysHlth as we discussed in initial data analysis. 

In [0]:
df = df.drop('AnyHealthcare','MentHlth','PhysHlth')
df.show()

+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-----------+-------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck|BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|NoDocbcCost|GenHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-----------+-------+--------+---+---+---------+------+
|           0|     0|       1|        1| 15|     1|     0|                   0|           0|     1|      1|                0|          0|      5|       0|  0| 11|        4|     5|
|           2|     1|       0|        1| 28|     0|     0|                   1|           0|     1|      0|                0|          0|      2|       0|  0| 11|        4|     3|
|           2|     1|       1|        1| 33|     0|     0|                   0|           1|     1| 

There are small precentage who are in the pre-diabetics region. Sience we want to train a model that predicts diabetics or not let's drop them

In [0]:
df_new = df.filter(df['Diabetes_012']!=1)
df_new.show()

+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-----------+-------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck|BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|NoDocbcCost|GenHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-----------+-------+--------+---+---+---------+------+
|           0|     0|       1|        1| 15|     1|     0|                   0|           0|     1|      1|                0|          0|      5|       0|  0| 11|        4|     5|
|           2|     1|       0|        1| 28|     0|     0|                   1|           0|     1|      0|                0|          0|      2|       0|  0| 11|        4|     3|
|           2|     1|       1|        1| 33|     0|     0|                   0|           1|     1| 

Since there are no nulls we doesn;t require to drop or fill null values using relevant value (like column mean). Therefore, Let's save dataframe as Parquet file and end session.


In [0]:
path = "dbfs:/FileStore/tables/fiterd_diabetics.parquet"
df_new.write.parquet(path)

In [0]:
spark.stop()